# Visualizing the logs

In [ ]:
import os
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json, ast
import matplotlib.dates as mdates
from dotenv import load_dotenv
load_dotenv()

kaffe_file = os.getenv("KAFFE_LOG_FILE")
lunsj_file = os.getenv("LUNSJ_LOG_FILE")
konge_file = os.getenv("KONGE_LOG_FILE")
bordtennis_file = os.getenv("BORDTENNIS_LOG_FILE")

try:
    env_map = os.getenv("MAP_ID_NAME", "{}") # should be on the form '{"<@123456789>": "name"}'
    MAP_ID_NAME = json.loads(env_map)
except Exception:
    try: 
        MAP_ID_NAME = ast.literal_eval(env_map)
    except Exception:
        MAP_ID_NAME = {}
        
def get_data(filename):
    with open(filename, "r") as file:
        csvdata = csv.reader(file)
        data = [row for row in csvdata]
        cols_name = data[0] # first row should be the column names
        rows = data[1:]
        return {title: d for title, d in zip(cols_name, zip(*rows))}

In [ ]:
def plot_timeseries_logs(filename):
    print(f"Plot of {filename}")
    data = get_data(filename)
    plot_title = filename.split("/")[1].split("_")[0] # given that we have the format logs/kaffe_logs.csv
    
    plt.style.use('_mpl-gallery')
    
    df_plot = pd.DataFrame({
           "timestamp": pd.to_datetime(list(data["timestamp"])),
           "duration": pd.to_numeric(list(data["duration"]), errors="coerce")
    }).dropna().sort_values("timestamp")
    
    x = df_plot["timestamp"]
    y = df_plot["duration"] / 60
    
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(x, y, linewidth=2)
    
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    fig.autofmt_xdate()
    ax.set_xlabel("timestamp")
    ax.set_ylabel("duration in minutes")
    ax.set_title(f"Duration of {plot_title} breaks")
    
    plt.show()

def plot_konge_logs(filename):
    print(f"Plot of {filename}")
    data = get_data(filename)

    plt.style.use('_mpl-gallery')
    
    get_names = [MAP_ID_NAME[str(id)] if id in MAP_ID_NAME else id for id in list(data["konge"]) ]
    get_counts = [get_names.count(name) for name in get_names]

    df_plot = pd.DataFrame({
           "konge": get_names,
           "count": get_counts
    }).dropna()

    x = df_plot["konge"]
    y = df_plot["count"]

    fig, ax = plt.subplots(figsize=(10,4))
    ax.bar(x, y, linewidth=20)

    ax.set_xlabel("konge")
    ax.set_ylabel("number of times")
    ax.set_title("Count of konge")
    
    plt.show()

In [ ]:
plot_timeseries_logs(kaffe_file)

In [ ]:
plot_timeseries_logs(lunsj_file)

In [ ]:
plot_timeseries_logs(bordtennis_file)

In [ ]:
plot_konge_logs(konge_file)